# 4. Optimizer 및 Training

## 4-1.Beginner
이전까지 layer써서 모델을 만들고 했는데 이 모델을 학습시키는 것을 배워보도록 한다. 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

### 1. prepare mnist datasets

In [2]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

### 2. build model

In [3]:
inputs = layers.Input((28, 28, 1))
net = layers.Conv2D(32, (3,3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)

net = layers.Conv2D(32, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)

net = layers.MaxPooling2D(pool_size = (2, 2))(net)
net = layers.Dropout(0.25)(net)

In [4]:
net = layers.Conv2D(64, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)

net = layers.Conv2D(64, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)

net = layers.MaxPooling2D(pool_size = (2, 2))(net)
net = layers.Dropout(0.25)(net)

In [5]:
net =layers.Flatten()(net)

net = layers.Dense(512)(net)

net = layers.Activation('relu')(net)

net = layers.Dropout(0.5)(net)

net = layers.Dense(10)(net)

net = layers.Activation('softmax')(net)

In [6]:
model = tf.keras.Model(inputs = inputs, outputs = net, name = 'BASIC_CNN')

### 3. Optimization : 모델을 학습 하기 전 설정하는 것들
- Loss function
- Optimization
- Metrics

#### 1) Loss Function
: cross entropy사용
    1. Categorical cross entropey가 있고,
        1) sparse_categoical_crossentropy **tf.keras.losses.sparse_categorical_crossentropy**
        2) categorical_crossentropt**tf.keras.losses.categorical_crossentropy**
    2. Binary cross entropy가 있다.  **tf.keras.losses.binary_crossentropy**

In [7]:
# 이 중 여기서는 sparse_categorical_crossentropy를 사용 할 것이다. 
loss_func = tf.keras.losses.sparse_categorical_crossentropy

#### 2) Metrics : 모델을 평가하는 방법
- accuracy(전체 문제에서 몇 개를 맞출 확률) 로 평가

In [8]:
metrics = ['accuracy'] #tf.keras.metrics.Accuracy()로 하는 방법도 있다. 

#### 3) Compile : Optimizer 적용
- 'sgd'
- 'rmsprop'
- 'adam'

In [9]:
#여기선 adam으로 적용
optm = tf.keras.optimizers.Adam()

In [10]:
model.compile(optimizer= optm, 
                  loss = loss_func, 
                  metrics = metrics) 
#optm = tf.keras.optimizers.Adam()
#loss_func = tf.keras.losses.sparse_categorical_crossentropy
#metrics = ['accuracy'] 

### 4. Prepare Dataset : 학습에 활용할 데이터셋 준비

#### 1) shape확인

In [11]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

#### 2) 차원 수 늘리기 : channel이 없어서 3차원을 4차원을 늘려줘야 함

In [20]:
import numpy as np

train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

#### 3) Rescaling : 이를 해 줘야 학습이 잘 된다. 

In [21]:
np.min(train_x), np.max(train_x)

(0.0, 1.0)

In [22]:
train_x = train_x/255.
test_x = test_x/255.

In [23]:
np.min(train_x), np.max(test_x)

(0.0, 0.00392156862745098)

### 5. Training : 본격적으로 학습 들어가기

#### 1) 학습용 Hyperparameter 설정
- num_epochs : 데이터를 한 번 쭉 보면 1 epoch 이다. (즉, 우리가 흔히 말하는 1회독 개념)
- batch_szie  : 한 epoch를 하기 위해서 batch_size만큼만 차례로 넣어준다. (데이터 막 몇 만개 되는데 한번에 넣어주면 메모리 효율이 떨어진다. )

In [24]:
num_epochs = 1
batch_size  = 32

#### 2) model fit

In [25]:
model.fit(train_x, train_y,
              batch_size = batch_size, 
             shuffle = True,  #데이터가 순서대로 들어가면 특정 숫자에 bias가 걸리게 된다. 그래서 랜덤하게 데이터를 섞어주는게 필요하다.
             epochs = num_epochs)

Train on 60000 samples
60000/60000 [==============================] - 157s 3ms/sample - loss: 2.3017 - accuracy: 0.1115
